<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# ============================================================
# Finite Element Analysis - 1D bar + spring (Problem 3)
# ============================================================

def solve_system(GDof, prescribed_dof, K, F, u_prescribed=None):
    """
    Solve KU = F with prescribed DOFs.
    prescribed_dof: list of 1-based dof indices (like MATLAB)
    """
    if u_prescribed is None:
        u_prescribed = np.zeros(len(prescribed_dof), dtype=float)

    # Convert to 0-based indices
    prescribed = np.array([d-1 for d in prescribed_dof], dtype=int)
    all_dofs = np.arange(GDof, dtype=int)
    free = np.setdiff1d(all_dofs, prescribed)

    U = np.zeros(GDof, dtype=float)
    U[prescribed] = u_prescribed

    # Partition
    K_ff = K[np.ix_(free, free)]
    K_fp = K[np.ix_(free, prescribed)]
    F_f  = F[free]

    # Solve for free DOFs: K_ff * U_f = F_f - K_fp * U_p
    rhs = F_f - K_fp @ U[prescribed]
    U[free] = np.linalg.solve(K_ff, rhs)

    # Reactions at prescribed DOFs: R_p = (K_pf U_f + K_pp U_p) - F_p
    K_pf = K[np.ix_(prescribed, free)]
    K_pp = K[np.ix_(prescribed, prescribed)]
    F_p  = F[prescribed]
    R_p = (K_pf @ U[free] + K_pp @ U[prescribed]) - F_p

    return U, R_p, free, prescribed

def output_displacements_reactions(U, R_p, prescribed_dof):
    print("=== NODAL DISPLACEMENTS (u) ===")
    for i, ui in enumerate(U, start=1):
        print(f"Node {i}: u = {ui:.6e}")

    print("\n=== REACTIONS AT PRESCRIBED DOFs ===")
    for dof_1based, Ri in zip(prescribed_dof, R_p):
        print(f"DOF {dof_1based} (Node {dof_1based}): R = {Ri:.6e}")

# -----------------------------
# INPUT (same as MATLAB)
# -----------------------------
E = 70000.0   # (N/mm^2) if using mm-N system
A = 200.0     # (mm^2)
k = 2000.0    # spring stiffness (N/mm) to be consistent with EA/L above

numberElements = 3
numberNodes = 4

elementNodes = np.array([
    [1, 2],
    [2, 3],
    [3, 4]
], dtype=int)

nodeCoordinates = np.array([0.0, 2000.0, 4000.0, 4000.0], dtype=float)  # mm
xx = nodeCoordinates.copy()

# Initialize
displacements = np.zeros(numberNodes, dtype=float)
force = np.zeros(numberNodes, dtype=float)
stiffness = np.zeros((numberNodes, numberNodes), dtype=float)

# Applied load at node 2 (same as MATLAB: force(2)=8000)
force[1] = 8000.0  # N

# -----------------------------
# Assemble global stiffness
# -----------------------------
ea = np.zeros(numberElements, dtype=float)

for e in range(numberElements):
    elementDof = elementNodes[e, :]              # 1-based node numbers
    n1, n2 = elementDof[0], elementDof[1]
    L = nodeCoordinates[n2-1] - nodeCoordinates[n1-1]

    if (e+1) < 3:  # MATLAB: if e<3 (with e starting at 1)
        ea[e] = E * A / L
    else:
        ea[e] = k

    # Local stiffness for 2-node 1D element: ea * [[1,-1],[-1,1]]
    k_local = ea[e] * np.array([[1.0, -1.0],
                                [-1.0,  1.0]])

    # Assemble into global stiffness (convert to 0-based indices)
    idx = np.array([n1-1, n2-1], dtype=int)
    stiffness[np.ix_(idx, idx)] += k_local

# -----------------------------
# Boundary conditions + solution
# -----------------------------
prescribedDof = [1, 4]  # fixed at node 1 and node 4 (MATLAB: [1;4])
GDof = numberNodes

U, R_p, free, prescribed = solve_system(GDof, prescribedDof, stiffness, force)

# Output
output_displacements_reactions(U, R_p, prescribedDof)

# Optional: show matrices
print("\n=== GLOBAL STIFFNESS MATRIX K ===")
print(stiffness)

print("\n=== GLOBAL FORCE VECTOR F ===")
print(force)


=== NODAL DISPLACEMENTS (u) ===
Node 1: u = 0.000000e+00
Node 2: u = 9.350649e-01
Node 3: u = 7.272727e-01
Node 4: u = 0.000000e+00

=== REACTIONS AT PRESCRIBED DOFs ===
DOF 1 (Node 1): R = -6.545455e+03
DOF 4 (Node 4): R = -1.454545e+03

=== GLOBAL STIFFNESS MATRIX K ===
[[ 7000. -7000.     0.     0.]
 [-7000. 14000. -7000.     0.]
 [    0. -7000.  9000. -2000.]
 [    0.     0. -2000.  2000.]]

=== GLOBAL FORCE VECTOR F ===
[   0. 8000.    0.    0.]
